### Walk-through of the Distance-Based SpSSIM Analysis & Python Script
**Jessica Embury, San Diego State University, Fall 2022**

Reference Literature: *Jin, C., Nara, A., Yang, J.-A., Tsou, M.-H. (2020). Similarity measurement on human mobility data with spatially weighted structural similarity index (SpSSIM). Transactions in GIS, 24, 104-122. [https://doi.org/10.1111/tgis.12590](https://doi.org/10.1111/tgis.12590)*

#### NOTE: Use the "sample_data" folder to follow along with the walk-through.

#### Explanation of Python files
- *util.py*: general functions that support the analysis, but are not specific to it
- *matrixes.py*: functions for the generation of analysis matrixes (flow probabilities, weights)
- *spssim_distbased_analysis.py*: functions for calculation of SpSSIM values
- *main.py*: file path variables and script to run the analysis

#### 1. Create a distance matrix with a row and column for every Census Block Group (CBG).
Each value is the distance between the row CBG and the column CBG in kilometers (km).
**Note:** This function uses a cross join and takes a long time to run!

In [14]:
import pandas as pd
from matrixes import create_distance_matrix

cbg_shapefile = './sample_data/source_data/CENSUS_BLOCKGROUPTIGER2010/CENSUS_BLOCKGROUPTIGER2010.shp'
bg_csv = '/sample_data/source_data/cbgs_2019.csv'
distance_matrix_csv = './sample_data/cbg_distance_matrix.csv'

distance_matrix = create_distance_matrix(cbg_shapefile, bg_csv, distance_matrix_csv)
distance_matrix.head(3)

,cbg_orig,60730001001,60730001002,60730002011,60730002021,60730002022,60730002023,60730003001,60730003002,60730003003,...,60730216002,60730218001,60730218002,60730219001,60730219002,60730220001,60730220002,60730221001,60730221002,60730221003
0,60730001001,0.0,0.0,0.0,0.4,0.0,0.5,0.9,1.4,1.7,...,7.4,6.6,5.6,8.9,9.3,10.6,11.3,40.9,40.0,38.6
1,60730001002,0.0,0.0,0.3,0.3,0.0,0.5,1.2,1.6,1.9,...,7.1,6.2,5.2,8.7,9.2,10.6,11.4,41.0,40.1,38.7
2,60730002011,0.0,0.3,0.0,0.1,0.0,0.0,0.1,0.5,0.8,...,7.5,6.8,5.7,8.5,8.8,9.9,10.6,41.3,40.4,39.1


#### 2. Download Origin-Destination (O-D) data and save as .CSV files.
These tables have a record for each origin, destination, and mobility value (e.g., job, device count). Below is a sample of the LODES 2019 O-D table.

In [15]:
od2017_table_csv = './sample_data/source_data/lodes_od_2017.csv'
od2017_df = pd.read_csv(od2017_table_csv)

od2019_table_csv = './sample_data/source_data/lodes_od_2019.csv'
od2019_df = pd.read_csv(od2019_table_csv)
print('The LODES 2019 O-D table has {} rows.'.format(len(od2019_df)))
od2019_df.head(3)

The LODES 2019 O-D table has 428273 rows.


,cbg_orig,cbg_dest,num_jobs
0,60730001001,60730001001,3
1,60730001001,60730001002,12
2,60730001001,60730002011,2


#### 3. Convert the O-D table into an O-D matrix with one row for each origin CBG and one column for each destination CBG.

In [16]:
from matrixes import odtable2matrix

od2017_raw_matrix_csv = './sample_data/spssim_distance_based/sd_lodes_2017_raw_matrix.csv'
od2017_matrix = odtable2matrix(od2017_table_csv, od2017_raw_matrix_csv, 'cbg_orig', 'cbg_dest', 'num_jobs')

od2019_raw_matrix_csv = './sample_data/spssim_distance_based/sd_lodes_2019_raw_matrix.csv'
od2019_matrix = odtable2matrix(od2019_table_csv, od2019_raw_matrix_csv, 'cbg_orig', 'cbg_dest', 'num_jobs')
od2019_matrix.head(3)

./sample_data/source_data/lodes_od_2017.csv
O-D table has 422719 rows.
Pivot table has 1794 rows and 1794 columns. 0 differences: []
./sample_data/source_data/lodes_od_2019.csv
O-D table has 428273 rows.
Pivot table has 1794 rows and 1794 columns. 0 differences: []


cbg_dest,60730001001,60730001002,60730002011,60730002021,60730002022,60730002023,60730003001,60730003002,60730003003,60730003004,...,60730216002,60730218001,60730218002,60730219001,60730219002,60730220001,60730220002,60730221001,60730221002,60730221003
cbg_orig,,,,,,,,,,,,,,,,,,,,,
60730001001,3.0,12.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0
60730001002,14.0,18.0,6.0,3.0,0.0,1.0,0.0,0.0,6.0,0.0,...,2.0,0.0,0.0,1.0,0.0,0.0,0.0,7.0,0.0,0.0
60730002011,0.0,0.0,37.0,0.0,1.0,0.0,0.0,2.0,3.0,1.0,...,0.0,2.0,0.0,14.0,1.0,0.0,0.0,2.0,0.0,0.0


#### 4. Convert the O-D matrix into a flow probabilities matrix with one row for each origin CBG and one column for each destination CBG.
The value is now the probability of the origin CBG row having flow into the destination CBG column. The sum of each row equals 1.

In [17]:
from matrixes import create_distbased_probability_matrix

flow_probs2017_matrix_csv = './sample_data/spssim_distance_based/sd_lodes_2017_distbased_flow_probabilities_matrix.csv'
flow_probs2017_matrix = create_distbased_probability_matrix(od2017_raw_matrix_csv, flow_probs2017_matrix_csv, 'cbg_orig')

flow_probs2019_matrix_csv = './sample_data/spssim_distance_based/sd_lodes_2019_distbased_flow_probabilities_matrix.csv'
flow_probs2019_matrix = create_distbased_probability_matrix(od2019_raw_matrix_csv, flow_probs2019_matrix_csv, 'cbg_orig')
flow_probs2019_matrix.head(3)

Raw matrix has 1794 rows. Probabilities matrix has 1794 rows.
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]
Raw matrix has 1794 rows. Probabilities matrix has 1794 rows.
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1.]


,60730001001,60730001002,60730002011,60730002021,60730002022,60730002023,60730003001,60730003002,60730003003,60730003004,...,60730218001,60730218002,60730219001,60730219002,60730220001,60730220002,60730221001,60730221002,60730221003,row_total
cbg_orig,,,,,,,,,,,,,,,,,,,,,
60730001001,0.000000,0.027523,0.004587,0.000000,0.00000,0.004587,0.0,0.000000,0.004587,0.00000,...,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.009174,0.004587,0.0,1.0
60730001002,0.025974,0.000000,0.011132,0.005566,0.00000,0.001855,0.0,0.000000,0.011132,0.00000,...,0.000000,0.0,0.001855,0.00000,0.0,0.0,0.012987,0.000000,0.0,1.0
60730002011,0.000000,0.000000,0.000000,0.000000,0.00135,0.000000,0.0,0.002699,0.004049,0.00135,...,0.002699,0.0,0.018893,0.00135,0.0,0.0,0.002699,0.000000,0.0,1.0


#### 5. Choose the distance bins.
Distance-based SpSSIMs will be calculated for the CBGs inside each distance bin. Below are 10km distance bins.


In [27]:
distance_bins = [(0, 10), (10, 20), (20, 30), (30, 40), (40, 50), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100), (100, 110), (110, 120)]

#### 6. Create a weight matrix for each distance bin.
Matrix cells equal 1 if the distance between the origin and destination CBGs is within the distance bin's range. Otherwise, the cell values equal 0.


In [28]:
from matrixes import create_weights_matrix

weights_matrix_csv = './sample_data/weights_10km_bins/weights_matrix_{}km_{}km.csv'
for b in distance_bins:
    df = create_weights_matrix(distance_matrix_csv, weights_matrix_csv, b)

df.head(3)

,60730001001,60730001002,60730002011,60730002021,60730002022,60730002023,60730003001,60730003002,60730003003,60730003004,...,60730216002,60730218001,60730218002,60730219001,60730219002,60730220001,60730220002,60730221001,60730221002,60730221003
cbg_orig,,,,,,,,,,,,,,,,,,,,,
60730001001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60730001002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60730002011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 7. Use 2 probability flow matrixes and the weights matrixes to calculate the SpSSIM.
Behind-the-scenes helper functions are used to calculate the mean and variance for each probability flow matrix, the covariance of the matrixes, and the local SpSSIM value for each distance bin. A results table with this data generated and saved.


In [31]:
from spssim_distbased_analysis import calc_global_distbased_spssim

results_csv = './sample_data/spssim_distance_based/spssim_results/spssim_distbased_results_lodes2017_lodes2019.csv'
spssim, results = calc_global_distbased_spssim(flow_probs2017_matrix_csv, flow_probs2019_matrix_csv, weights_matrix_csv, results_csv, 'cbg_orig', distance_bins)

./sample_data/sd_lodes_2017_flow_probabilities_matrix.csv ./sample_data/sd_lodes_2019_flow_probabilities_matrix.csv 0.5507424989396075


#### In this example, I calculated the similarity between 2017 LODES data and 2019 LODES data.
SpSSIM scores range from 0-1 with a score equal to 1 if the flow probabilities matrixes are the same.


In [32]:
print('The SpSSIM value is {}.'.format(spssim))
results

The SpSSIM value is 0.5507424989396075.


,matrix1,matrix2,distance_bin,constant1,constant2,n,mean1,mean2,variance1,variance2,covariance,distbased_spssim,global_spssim
0,sample_data/sd_lodes_2017_flow_probabilities_m...,sample_data/sd_lodes_2019_flow_probabilities_m...,"(0, 10)",0,0,3218436,2.525394e-04,2.528100e-04,6.109287e-06,6.210212e-06,5.578437e-06,9.056266e-01,0.550742
1,sample_data/sd_lodes_2017_flow_probabilities_m...,sample_data/sd_lodes_2019_flow_probabilities_m...,"(10, 20)",0,0,3218436,1.664072e-04,1.663069e-04,2.703686e-06,2.759377e-06,2.363007e-06,8.650848e-01,0.550742
2,sample_data/sd_lodes_2017_flow_probabilities_m...,sample_data/sd_lodes_2019_flow_probabilities_m...,"(20, 30)",0,0,3218436,7.961712e-05,7.943026e-05,9.366678e-07,9.563944e-07,7.774227e-07,8.213366e-01,0.550742
3,sample_data/sd_lodes_2017_flow_probabilities_m...,sample_data/sd_lodes_2019_flow_probabilities_m...,"(30, 40)",0,0,3218436,2.960345e-05,2.969408e-05,2.580090e-07,2.699642e-07,2.001532e-07,7.581912e-01,0.550742
4,sample_data/sd_lodes_2017_flow_probabilities_m...,sample_data/sd_lodes_2019_flow_probabilities_m...,"(40, 50)",0,0,3218436,1.608152e-05,1.594261e-05,9.247835e-08,9.077109e-08,5.651546e-08,6.167913e-01,0.550742
5,sample_data/sd_lodes_2017_flow_probabilities_m...,sample_data/sd_lodes_2019_flow_probabilities_m...,"(50, 60)",0,0,3218436,8.382213e-06,8.436751e-06,4.641637e-08,4.304496e-08,2.598215e-08,5.808454e-01,0.550742
6,sample_data/sd_lodes_2017_flow_probabilities_m...,sample_data/sd_lodes_2019_flow_probabilities_m...,"(60, 70)",0,0,3218436,3.312648e-06,3.394302e-06,1.375305e-08,1.452843e-08,5.421036e-09,3.832494e-01,0.550742
7,sample_data/sd_lodes_2017_flow_probabilities_m...,sample_data/sd_lodes_2019_flow_probabilities_m...,"(70, 80)",0,0,3218436,1.078548e-06,1.034008e-06,4.379966e-09,3.459700e-09,9.008138e-10,2.296050e-01,0.550742
8,sample_data/sd_lodes_2017_flow_probabilities_m...,sample_data/sd_lodes_2019_flow_probabilities_m...,"(80, 90)",0,0,3218436,3.040730e-07,2.676837e-07,4.823677e-09,3.318812e-09,3.094420e-09,7.539348e-01,0.550742
9,sample_data/sd_lodes_2017_flow_probabilities_m...,sample_data/sd_lodes_2019_flow_probabilities_m...,"(90, 100)",0,0,3218436,7.782013e-08,8.819877e-08,3.982667e-10,5.576804e-10,1.111882e-10,2.308130e-01,0.550742


#### 8. Calculate SpSSIM constants and re-run the analysis.
The first iteration of the analysis uses constants equal to zero. After finding all SpSSIM values, calculate the constants such that the least similar matrix pairing has an SpSSIM equal to zero. (The constants adjust the equations so that SpSSIM scores are not negative.)

In [33]:
from spssim_distbased_analysis import calc_distbased_spssim_constants

results_directory = ['./sample_data/spssim_distance_based/spssim_results/']
constant1, constant2 = calc_distbased_spssim_constants(results_directory)
print('C1 = {}, C2 = {}'.format(constant1, constant2))

Minimum distbased SpSSIM = -3.13475145332376e-07
Updated minimum distbased SpSSIM = 0.0
C1 = -1.232301941156727e-17, C2 = 1.2323023240453003e-17


#### Next, re-run the analysis using the updated constants.

In [34]:
results_csv = './sample_data/spssim_distance_based/spssim_results/spssim_results_lodes2017_lodes2019.csv'
spssim, results = calc_global_distbased_spssim(flow_probs2017_matrix_csv, flow_probs2019_matrix_csv, weights_matrix_csv, results_csv, 'cbg_orig', distance_bins, constant1, constant2)

print('The SpSSIM value is {}.'.format(spssim))
results.head(3)

./sample_data/sd_lodes_2017_flow_probabilities_matrix.csv ./sample_data/sd_lodes_2019_flow_probabilities_matrix.csv 0.5507425339213279
The SpSSIM value is 0.5507425339213279.


,matrix1,matrix2,distance_bin,constant1,constant2,n,mean1,mean2,variance1,variance2,covariance,distbased_spssim,global_spssim
0,sample_data/sd_lodes_2017_flow_probabilities_m...,sample_data/sd_lodes_2019_flow_probabilities_m...,"(0, 10)",0,1.232302e-17,3218436,0.000253,0.000253,6.109287e-06,6.210212e-06,5.578437e-06,0.905627,0.550743
1,sample_data/sd_lodes_2017_flow_probabilities_m...,sample_data/sd_lodes_2019_flow_probabilities_m...,"(10, 20)",0,1.232302e-17,3218436,0.000166,0.000166,2.703686e-06,2.759377e-06,2.363007e-06,0.865085,0.550743
2,sample_data/sd_lodes_2017_flow_probabilities_m...,sample_data/sd_lodes_2019_flow_probabilities_m...,"(20, 30)",0,1.232302e-17,3218436,0.000080,0.000079,9.366678e-07,9.563944e-07,7.774227e-07,0.821337,0.550743


#### 9. Finally, generate summaries of the global SpSSIM and local SpSSIM results.

In [36]:
from spssim_distbased_analysis import calc_distbased_results_summaries

results_directory = './sample_data/spssim_distance_based/spssim_results/'
global_summ_csv = './sample_data/spssim_distance_based/global_summary.csv'
distbased_summ_csv = './sample_data/spssim_distance_based/distbased_summary.csv'
global_summ, distbased_summ = calc_distbased_results_summaries(results_directory, global_summ_csv, distbased_summ_csv)

global_summ.head(3)

,global_spssim,matrix_pair
0,0.550743,spssim_results_lodes2017_lodes2019


In [37]:
distbased_summ.head(3)

,distance_bin,mean_distbased_spssim,min_distbased_spssim,min_matrix_pair,max_distbased_spssim,max_matrix_pair,distbased_spssim_list,matrix_pairs
0,"(0, 10)",0.905627,0.905627,spssim_results_lodes2017_lodes2019,0.905627,spssim_results_lodes2017_lodes2019,[0.905626611568666],[spssim_results_lodes2017_lodes2019]
1,"(10, 20)",0.865085,0.865085,spssim_results_lodes2017_lodes2019,0.865085,spssim_results_lodes2017_lodes2019,[0.8650847698576077],[spssim_results_lodes2017_lodes2019]
2,"(20, 30)",0.821337,0.821337,spssim_results_lodes2017_lodes2019,0.821337,spssim_results_lodes2017_lodes2019,[0.8213365566574209],[spssim_results_lodes2017_lodes2019]


#### The Python script is available on [GitHub](https://github.com/jlembury/spssim_analysis). Backup data is on [Google Drive](https://drive.google.com/drive/folders/12WrJ_iIWFP6eIQUoisutON2G8xudTM-D?usp=sharing).
